This notebook demonstrates a basic **GT simulation** workflow for numerically integrating the equation of motion of a charged particle in the Heliosphere with the inclusion of adiabatic losses of energy.

Goals of this example:
- Define the `Parker` magnetic field and initial conditions for protons with kinetic energy in range between $320\div 32000$ MeV, from starting from $(1, 0, 0)$ AU
- Compute the trajectory with Buneman–Boris (BB) and an energy loss due to adiabatic losses

Expected output:
- A plot of energy loss dependence over time for protons of considered initial energies

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':15, 'figure.figsize':(15, 10)})
import numpy as np
from gtsimulation.Algos import BunemanBorisSimulator
from gtsimulation.Global import Regions, Units
from gtsimulation.MagneticFields.Heliosphere import Parker
from gtsimulation.Particle import Generators, Flux


# Problem setup: field, particle, and integration parameters

This section defines the simulation setup:
- `date` (used by time-dependent field models);
- the `Heliosphere` region and the `Parker` field model;
- in region parameters if we activate *additional energy losses*
- particle initial conditions: 10 batches of 10 protons in energy range $320\div 32000$ MeV with an initial coordinate $(1, 0, 0)$ AU.
- integration settings: number of steps `n_steps`, and time step `dt`;
- which quantities to store in the track (coordinates, velocities, energy, and time along the trajectory).
- simulation using BB scheme

In [ ]:
date = datetime(2008, 1, 1)
region = Regions.Heliosphere
region.value.set_params(CalcAdditionalEnergy=True) # this line of code includes the adiabatic energy losses into the calculation
bfield = Parker()

medium = None
use_decay = False
nuclear_interaction = None

flux = Flux(
    Spectrum=Generators.Spectrums.UserInput(energy=np.logspace(2.5, 4.5, 10, endpoint=True)*Units.MeV),
    Distribution=Generators.Distributions.SphereSurf(Center=np.array([1, 0, 0])*Units.AU, Radius=0),
    Names="proton",
    Nevents=10,
)

save = [100, {'Energy': True, 'Clock': True}]
steps = 1000000
dt = 0.1

forward_tracking = 1
nfiles = 10
output = None

break_conditions = {"Rmax": 80*Units.AU}
verbose = True

In [ ]:
simulator_BB = BunemanBorisSimulator(
    Bfield=bfield,
    Region=region,
    Medium=medium,
    Particles=flux,
    InteractNUC=nuclear_interaction,
    UseDecay=use_decay,
    Date=date,
    Step=dt,
    Num=steps,
    Nfiles=nfiles,
    BreakCondition=break_conditions,
    Save=save,
    Output=output,
    ForwardTrck=forward_tracking,
    Verbose=verbose
)

tracks = simulator_BB()

# Energy loss vs time

Here we plot the relative energy loss $\left(-\dfrac{\Delta T}{T_0}\right)$ dependence on time for protons with considered initial energies (Fig. 19 in the paper takes every second energy value).

We have an ensemble of initial conditions, as we considered 10 batches of particles. This allows to estimate the median relative energy loss (the solid line) and the $15\% \div 85\%$, i.e. $1\sigma$ spread (the shaded area).

In [ ]:
adiabatic_losses = {}
time_energy = {}

for events in tracks:
    for event in events:
        T0 = event["Particle"]["T0"]
        E = event["Track"]["Energy"]
        t = event["Track"]["Clock"]

        if T0 in adiabatic_losses:
            adiabatic_losses[T0].append(E)
        else:
            adiabatic_losses[T0] = [E]
        time_energy[T0] = (np.max(t), len(t)) if T0 not in time_energy else (min(np.max(t), time_energy[T0][0]),
                                                                         min(len(t), time_energy[T0][1]))

plt.figure()

for T in list(adiabatic_losses.keys()):
    max_t, lent = time_energy[T]
    time = np.linspace(0, max_t, lent, endpoint=True)
    En = []
    for arr in adiabatic_losses[T]:
        En.append(arr[:lent])
    En = np.array(En)
    med = np.median(En, axis=0)
    low = np.quantile(En, q=0.15, axis=0)
    high = np.quantile(En, q=0.85, axis=0)
    plt.plot((time/(3600*24))[time/(3600*24)>2e-3],
             (T-med)[time/(3600*24)>2e-3]/T,
             label=f'T = {np.round(T, -1).astype(int)} MeV')
    plt.fill_between((time/(3600*24))[time/(3600*24)>2e-3],
                     y1=(T-low)[time/(3600*24)>2e-3]/T,
                     y2=(T-high)[time/(3600*24)>2e-3]/T,
                     alpha=0.5)

plt.xscale('log')
plt.ylabel(r"$\frac{-\Delta T}{T_0}$")
plt.xlabel('time [days]')
plt.grid(which='both')
plt.legend()
plt.show()